In [17]:
import pandas as pd
import copy

In [4]:
# This time only going to define bounds
{'a': (0, 10),
'b': (0, 100)}

{'a': (0, 10), 'b': (0, 100)}

In [21]:
def get_initial_coords(bounds):
    """ Function to get initial coordinates to tune """

    # Algorithm to create all bounds
    init_coords = [[bounds[0][i]] for i in range(2)]
    for i in range(1, len(bounds)):
        old_init_coords = copy.deepcopy(init_coords)
        init_coords = list()

        values = bounds[i]

        for init_coord in old_init_coords:
            for value in values:
                tmp = copy.deepcopy(init_coord)
                tmp.append(value)

                init_coords.append(tmp)
    
    return init_coords

In [31]:
def get_new_bounds(bounds, centre):
    """ Function to get new bounds """

    # get the range components that make the 2^d bounds
    range_components = get_range_components(bounds, centre)

    # make the new bounds from components
    new_bounds = make_bounds(range_components)

    return new_bounds



def get_range_components(bounds, centre):
    """ Helper that gets the range components """

    range_components = list()
    for i in range(len(bounds)):

        lower_range = (bounds[i][0], centre[i]) 
        upper_range = (centre[i], bounds[i][1]) 

        ranges = (lower_range, upper_range)

        range_components.append(ranges)
    
    return range_components

    

def make_bounds(range_components):
    """ Helper that makes the bounds using range components """

    # Algorithm to create all bounds
    bounds = [[range_components[0][i]] for i in range(2)] # hardcode cos bounds can only have 2 values
    for i in range(1, len(range_components)):
        old_bounds = copy.deepcopy(bounds)
        bounds = list()

        values = range_components[i]

        for bound in old_bounds:
            for value in values:
                tmp = copy.deepcopy(bound)
                tmp.append(value)

                bounds.append(tmp)
    
    return bounds
        


def get_centre(bounds):
    """ Helper that gets the centre of a bound """

    centre = list()
    for bound in bounds:
        centre.append(sum(bound)/2)
    
    return centre

In [34]:
bounds = [(0, 10), (0, 100)]

centre = get_centre(bounds)   

In [35]:
get_new_bounds(bounds, centre)

[[(0, 5.0), (0, 50.0), (0, 25.0)],
 [(0, 5.0), (0, 50.0), (25.0, 50)],
 [(0, 5.0), (50.0, 100), (0, 25.0)],
 [(0, 5.0), (50.0, 100), (25.0, 50)],
 [(5.0, 10), (0, 50.0), (0, 25.0)],
 [(5.0, 10), (0, 50.0), (25.0, 50)],
 [(5.0, 10), (50.0, 100), (0, 25.0)],
 [(5.0, 10), (50.0, 100), (25.0, 50)]]